# Configuracion Inicial

### Importar Librerias

In [1]:
import os
from dotenv import load_dotenv          # PARA VARIABLES DE ENTORNO
import pandas as pd                     # PARA MANIPULACION DE ARCHIVOS
import pyodbc                           # PARA CONEXION CON BASE DE DATOS
import numpy as np                      # PARA CALCULOS MATEMATICOS
import seaborn as sns                   # PARA GRAFICAS ATRACTIVAS E INFORMATIVAS
import matplotlib.pyplot as plt         # PARA GRAFICOS

### Carga de Variables de Entorno

In [2]:
load_dotenv()

DBHOST = os.getenv("DB_HOST")
DBNAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_TABLE = os.getenv("DB_TABLE")

### Conexion con base de datos

In [3]:
connection = pyodbc.connect(
    f'DRIVER={{ODBC Driver 17 for SQL Server}};'
    f'SERVER={DBHOST};'
    f'DATABASE={DBNAME};'
    f'UID={DB_USER};'
    f'PWD={DB_PASSWORD}'
)
cursor = connection.cursor()
print("Conexión exitosa a SQL Server")

Conexión exitosa a SQL Server


# 1. Preparación de datos: 

### a. Extraer los datos del archivo .csv.

In [ ]:
df = pd.read_csv("archivo.csv") #nombre de nuestro archivo de carga
df.head()

,order_id,purchase_date,customer_id,customer_gender,customer_age,product_category,product_name,product_price,quantity,order_total,payment_method,shipping_region
0,96,8/12/23,822,Masculino,18.0,Accesorios,Bolso,90.72,3,272.16,PayPal,Oeste
1,142,3/09/23,139,Femenino,18.0,Ropa,Pantalón,131.68,3,395.04,PayPal,Oeste
2,216,24/08/23,86,Masculino,18.0,Calzado,Zapatillas,36.18,1,36.18,Transferencia bancaria,Este
3,235,15/03/23,473,Masculino,18.0,Ropa,Vestido,78.55,5,392.75,Transferencia bancaria,Centro
4,284,24/04/23,113,Masculino,18.0,Calzado,Sandalias,116.37,1,116.37,Tarjeta de crédito,Norte


### b. Verificar si hay valores faltantes o duplicados y decidir cómo manejarlos.

In [ ]:
print("Valores faltantes por columna:")
print(df.isnull().sum())

# Filas duplicadas
print("Número de filas duplicadas:", df.duplicated().sum())

# eliminamos filas con valores faltantes y duplicados
df = df.dropna()
df = df.drop_duplicates()

Valores faltantes por columna:
order_id             0
purchase_date        0
customer_id          0
customer_gender      4
customer_age         2
product_category     2
product_name        16
product_price        1
quantity             0
order_total          0
payment_method       2
shipping_region      0
dtype: int64
Número de filas duplicadas: 2


### c. Asegurarse de que los tipos de datos sean correctos para cada columna.

In [6]:
df['order_id'] = df['order_id'].astype(str)
df['purchase_date'] = pd.to_datetime(df['purchase_date'], errors='coerce')
df['customer_id'] = df['customer_id'].astype(str)
df['customer_gender'] = df['customer_gender'].astype('category')
df['customer_age'] = pd.to_numeric(df['customer_age'], errors='coerce')
df['product_category'] = df['product_category'].astype('category')
df['product_name'] = df['product_name'].astype(str)
df['product_price'] = pd.to_numeric(df['product_price'], errors='coerce')
df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce')
df['order_total'] = pd.to_numeric(df['order_total'], errors='coerce')
df['payment_method'] = df['payment_method'].astype('category')
df['shipping_region'] = df['shipping_region'].astype('category')

# Revisar tipos
df.dtypes


C:\Users\cutza\AppData\Local\Temp\ipykernel_19692\1039086035.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['purchase_date'] = pd.to_datetime(df['purchase_date'], errors='coerce')


order_id                    object
purchase_date       datetime64[ns]
customer_id                 object
customer_gender           category
customer_age               float64
product_category          category
product_name                object
product_price              float64
quantity                     int64
order_total                float64
payment_method            category
shipping_region           category
dtype: object

### d. Cargar los datos a una base de datos SQL en la nube.

In [ ]:
# Crear tabla si no existe
create_table_query = f"""
IF OBJECT_ID('{DB_TABLE}', 'U') IS NULL
CREATE TABLE {DB_TABLE} (
    order_id NVARCHAR(50),
    purchase_date DATETIME,
    customer_id NVARCHAR(50),
    customer_gender NVARCHAR(10),
    customer_age INT,
    product_category NVARCHAR(50),
    product_name NVARCHAR(100),
    product_price FLOAT,
    quantity INT,
    order_total FLOAT,
    payment_method NVARCHAR(50),
    shipping_region NVARCHAR(50)
)
"""
cursor.execute(create_table_query)
connection.commit()

# carga masiva de los datos
cursor.fast_executemany = True

insert_query = f"""
INSERT INTO {DB_TABLE} (
    order_id, purchase_date, customer_id, customer_gender,
    customer_age, product_category, product_name,
    product_price, quantity, order_total,
    payment_method, shipping_region
) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
"""

# Convertir DataFrame a lista de tuplas
data_tuples = list(df.itertuples(index=False, name=None))

# Insertar todos los datos de una sola vez
cursor.executemany(insert_query, data_tuples)
connection.commit()

print(f"Datos cargados correctamente '{DB_TABLE}'")


Datos cargados correctamente a la tabla 'order_data'


# 2. Analisis Exploratorio

### a. Obtener los datos de la base de datos. 

### b. Calcular estadísticas básicas (media, mediana, moda) para las variables numéricas.

### c. Crear visualizaciones para mostrar la distribución de ventas por categoría de producto y región. 

# 3. Análisis de tendencias:

### a. Determinar los meses con mayores y menores ventas.

### b. Identificar los productos más vendidos y los menos populares. 

# 4. Segmentación de clientes:

### a. Agrupar a los clientes por edad y analizar sus patrones de compra.

### b. Comparar el comportamiento de compra entre géneros.

# 5. Análisis de correlación: 

### a. Investigar si existe una relación entre el total de la orden y la edad del cliente.

### b. Examinar si hay una correlación entre la categoría del producto y el método de pago preferido.

# 6. Visualización de datos: 

### a. Crear al menos siete gráficos diferentes (por ejemplo, gráfico de barras, gráfico de dispersión, gráfico de líneas) para representar los hallazgos más importantes.

# 7. Conclusiones y recomendaciones: 

### a. Basándose en el análisis realizado, proporcionar al menos cuatro conclusiones clave sobre las ventas y el comportamiento de los clientes.

### b. Sugerir dos acciones concretas por estudiante que la empresa podría tomar para mejorar sus ventas o la satisfacción del cliente.

# 8. Responder a las preguntas:

### a. ¿Cómo podrían los insights obtenidos ayudar a diferenciarse de la competencia?

### b. ¿Qué decisiones estratégicas podrían tomarse basándose en este análisis para aumentar las ventas y la satisfacción del cliente?

### c. ¿Cómo podría este análisis de datos ayudar a la empresa a ahorrar costos o mejorar la eficiencia operativa?

### d. ¿Qué datos adicionales recomendarían recopilar para obtener insights aún más valiosos en el futuro?